<span style="font-size:2.5em; color:RoyalBlue">CS461 Project: Student Performance</span>
<br><br>

# Imports

In [75]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [76]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# I. Data Munging

## Import Pandas, and import the data as a dataframe.

In [77]:
import pandas as pd
df = pd.read_csv('student-mat.csv', sep=';')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


## Convert non-Numerical columns to Numerical

In [78]:
# for Sex replace F and M by 0 and 1
if(df.sex.dtype == 'object'):
    df.sex = df.sex.map({'F': 0, 'M':1})

# for Family Size replace LE3 and GT3 by 0 and 1
if(df.famsize.dtype == 'object'):
    df.famsize = df.famsize.map({'LE3': 0, 'GT3':1})
    
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,0,18,U,1,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,0,17,U,1,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,0,15,U,0,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,0,15,U,1,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,0,16,U,1,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [79]:
# dataFrameStr is a column of a DataFrame object
def convertToInt_YesNo(dataFrameStr):
    if(dataFrameStr.dtype == 'object'):
        return dataFrameStr.map({'no': 0, 'yes':1})
    else:
        return dataFrameStr

# dataFrameStr is a column of a DataFrame object
def convertToInt(dataFrameStr):
    uniqueValues = []
    [uniqueValues.append(item) for item in pd.unique(dataFrameStr)]

    intList = []    
    [intList.append(uniqueValues.index(item)) for item in dataFrameStr]
        
    return intList

In [80]:
df.school = convertToInt(df.school)
df.address = convertToInt(df.address)
df.Pstatus = convertToInt(df.Pstatus)
df.Mjob = convertToInt(df.Mjob)
df.Fjob = convertToInt(df.Fjob)
df.reason = convertToInt(df.reason)
df.guardian = convertToInt(df.guardian)
df.schoolsup = convertToInt_YesNo(df.schoolsup)
df.famsup = convertToInt_YesNo(df.famsup)
df.paid = convertToInt_YesNo(df.paid)
df.activities = convertToInt_YesNo(df.activities)
df.nursery = convertToInt_YesNo(df.nursery)
df.higher = convertToInt_YesNo(df.higher)
df.internet = convertToInt_YesNo(df.internet)
df.romantic = convertToInt_YesNo(df.romantic)
df.school = convertToInt_YesNo(df.school)

df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,18,0,1,0,4,4,0,0,...,4,3,4,1,1,3,6,5,6,6
1,0,0,17,0,1,1,1,1,0,1,...,5,3,3,1,1,3,4,5,5,6
2,0,0,15,0,0,1,1,1,0,1,...,4,3,2,2,3,3,10,7,8,10
3,0,0,15,0,1,1,4,2,1,2,...,3,2,2,1,1,5,2,15,14,15
4,0,0,16,0,1,1,3,3,2,1,...,4,3,2,1,2,5,4,6,10,10


In [81]:
df.dtypes

school        int64
sex           int64
age           int64
address       int64
famsize       int64
Pstatus       int64
Medu          int64
Fedu          int64
Mjob          int64
Fjob          int64
reason        int64
guardian      int64
traveltime    int64
studytime     int64
failures      int64
schoolsup     int64
famsup        int64
paid          int64
activities    int64
nursery       int64
higher        int64
internet      int64
romantic      int64
famrel        int64
freetime      int64
goout         int64
Dalc          int64
Walc          int64
health        int64
absences      int64
G1            int64
G2            int64
G3            int64
dtype: object

## Create a new column 'G3Class' for classifying G3 in 2 groups of 0's and 1's for those having G3 0-10 and 11-20 respectively

In [82]:
df['G3Class'] = df.G3
def classification(num):
    if(num <= 10):
        return 0
    else:
        return 1
        
df.G3Class = df.G3Class.map(lambda x: classification(x))
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G3Class
0,0,0,18,0,1,0,4,4,0,0,...,3,4,1,1,3,6,5,6,6,0
1,0,0,17,0,1,1,1,1,0,1,...,3,3,1,1,3,4,5,5,6,0
2,0,0,15,0,0,1,1,1,0,1,...,3,2,2,3,3,10,7,8,10,0
3,0,0,15,0,1,1,4,2,1,2,...,2,2,1,1,5,2,15,14,15,1
4,0,0,16,0,1,1,3,3,2,1,...,3,2,1,2,5,4,6,10,10,0


# II. Feature Engineering

## Create New Features 

In [83]:
features = df.shape[1]
for i in range(features):
    unique_values = pd.unique(df.iloc[:,i].ravel())
    header = df.columns.values[i]
    print header, unique_values

school [0 1]
sex [0 1]
age [18 17 15 16 19 22 20 21]
address [0 1]
famsize [1 0]
Pstatus [0 1]
Medu [4 1 3 2 0]
Fedu [4 1 2 3 0]
Mjob [0 1 2 3 4]
Fjob [0 1 2 3 4]
reason [0 1 2 3]
guardian [0 1 2]
traveltime [2 1 3 4]
studytime [2 3 1 4]
failures [0 3 2 1]
schoolsup [1 0]
famsup [0 1]
paid [0 1]
activities [0 1]
nursery [1 0]
higher [1 0]
internet [0 1]
romantic [0 1]
famrel [4 5 3 1 2]
freetime [3 2 4 1 5]
goout [4 3 2 1 5]
Dalc [1 2 5 3 4]
Walc [1 3 2 4 5]
health [3 5 1 2 4]
absences [ 6  4 10  2  0 16 14  7  8 25 12 54 18 26 20 56 24 28  5 13 15 22  3 21  1
 75 30 19  9 11 38 40 23 17]
G1 [ 5  7 15  6 12 16 14 10 13  8 11  9 17 19 18  4  3]
G2 [ 6  5  8 14 10 15 12 18 16 13  9 11  7 19 17  4  0]
G3 [ 6 10 15 11 19  9 12 14 16  5  8 17 18 13 20  7  0  4]
G3Class [0 1]


### Create catigorical binary-features from existing features with more than 2 categorical values

In [84]:
# Convert Feature: Medu
df['Medu_0'] = (df['Medu'] == 0).astype(int)
df['Medu_1'] = (df['Medu'] == 1).astype(int)
df['Medu_2'] = (df['Medu'] == 2).astype(int)
df['Medu_3'] = (df['Medu'] == 3).astype(int)
df['Medu_4'] = (df['Medu'] == 4).astype(int)

In [85]:
# Convert Feature: Fedu
df['Fedu_0'] = (df['Fedu'] == 0).astype(int)
df['Fedu_1'] = (df['Fedu'] == 1).astype(int)
df['Fedu_2'] = (df['Fedu'] == 2).astype(int)
df['Fedu_3'] = (df['Fedu'] == 3).astype(int)
df['Fedu_4'] = (df['Fedu'] == 4).astype(int)

In [86]:
# Convert Feature: Mjob
df['Mjob_0'] = (df['Mjob'] == 0).astype(int)
df['Mjob_1'] = (df['Mjob'] == 1).astype(int)
df['Mjob_2'] = (df['Mjob'] == 2).astype(int)
df['Mjob_3'] = (df['Mjob'] == 3).astype(int)
df['Mjob_4'] = (df['Mjob'] == 4).astype(int)

In [87]:
# Convert Feature: Fjob
df['Fjob_0'] = (df['Fjob'] == 0).astype(int)
df['Fjob_1'] = (df['Fjob'] == 1).astype(int)
df['Fjob_2'] = (df['Fjob'] == 2).astype(int)
df['Fjob_3'] = (df['Fjob'] == 3).astype(int)
df['Fjob_4'] = (df['Fjob'] == 4).astype(int)

In [88]:
# Convert Feature: Reason
df['reason_0'] = (df['reason'] == 0).astype(int)
df['reason_1'] = (df['reason'] == 1).astype(int)
df['reason_2'] = (df['reason'] == 2).astype(int)
df['reason_3'] = (df['reason'] == 3).astype(int)

In [89]:
# Convert Feature: guardian
df['gaurdian_0'] = (df['guardian'] == 0).astype(int)
df['gaurdian_1'] = (df['guardian'] == 1).astype(int)
df['gaurdian_2'] = (df['guardian'] == 2).astype(int)

## Select Features

In [90]:
# function: scale_features
# input: dataframe containing feature values
# output: dataframe containing normalized features

def scale_features(df):
    mean = df.mean(axis = 0)
    std = df.std(axis = 0)
    return (df - mean) / std

In [91]:
df_features = df.copy()

# delete duplicate features and feature-labels
del df_features['Fedu']
del df_features['Medu']
del df_features['Fjob']
del df_features['Mjob']
del df_features['reason']
del df_features['guardian']
del df_features['G1']
del df_features['G2']
del df_features['G3']
del df_features['G3Class']

In [92]:
df_features.dtypes

school        int64
sex           int64
age           int64
address       int64
famsize       int64
Pstatus       int64
traveltime    int64
studytime     int64
failures      int64
schoolsup     int64
famsup        int64
paid          int64
activities    int64
nursery       int64
higher        int64
internet      int64
romantic      int64
famrel        int64
freetime      int64
goout         int64
Dalc          int64
Walc          int64
health        int64
absences      int64
Medu_0        int64
Medu_1        int64
Medu_2        int64
Medu_3        int64
Medu_4        int64
Fedu_0        int64
Fedu_1        int64
Fedu_2        int64
Fedu_3        int64
Fedu_4        int64
Mjob_0        int64
Mjob_1        int64
Mjob_2        int64
Mjob_3        int64
Mjob_4        int64
Fjob_0        int64
Fjob_1        int64
Fjob_2        int64
Fjob_3        int64
Fjob_4        int64
reason_0      int64
reason_1      int64
reason_2      int64
reason_3      int64
gaurdian_0    int64
gaurdian_1    int64


## Normalize Features

In [93]:
# scale features
df_features = scale_features(df_features)

# III. Machine Learning

## Convert to Numpy Arrays

In [94]:
features = df_features.values
labels_regression = df['G3'].values
labels_classification = df['G3Class'].values

# add x0 feature column of ones to features matrix
rows = features.shape[0]
features = np.column_stack((np.ones((rows,1)), features))

## Classification

In [95]:
# split data
X_train, X_test, y_train, y_test = \
    train_test_split(features, labels_classification, test_size=0.33, random_state=0)

### Naive Bayes

In [96]:
# create model
clf = GaussianNB()

# fit model
clf.fit(X_train, y_train)

# predict
pred = clf.predict(X_test)

# accuracy
accuracy = metrics.accuracy_score(y_test, pred)

# print results
print "Gaussian Naive Bayes accuracy = ", accuracy

Gaussian Naive Bayes accuracy =  0.549618320611


### SVM

In [97]:
# create model
clf = SVC()

# fit model
clf.fit(X_train, y_train)

# predict
pred = clf.predict(X_test)

# accuracy
accuracy = metrics.accuracy_score(y_test, pred)

# print results
print "Support Vector Machine accuracy = ", accuracy

Support Vector Machine accuracy =  0.603053435115


### Logistic Regression

In [98]:
# create model
clf = LogisticRegression()

# fit model
clf.fit(X_train, y_train)

# predict
pred = clf.predict(X_test)

# accuracy
accuracy = metrics.accuracy_score(y_test, pred)

# print results
print "Logistic Regression accuracy = ", accuracy

Logistic Regression accuracy =  0.580152671756


### Random Forest

In [99]:
# create model
clf = RandomForestClassifier(n_estimators = 100)

# fit model
clf.fit(X_train, y_train)

# predict
pred = clf.predict(X_test)

# accuracy
accuracy = metrics.accuracy_score(y_test, pred)

# print results
print "Random Forest accuracy = ", accuracy

Random Forest accuracy =  0.641221374046


### Decision Tree

In [100]:
# create model
clf = DecisionTreeClassifier()

# fit model
clf.fit(X_train, y_train)

# predict
pred = clf.predict(X_test)

# accuracy
accuracy = metrics.accuracy_score(y_test, pred)

# print results
print "Decision Tree accuracy = ", accuracy

Decision Tree accuracy =  0.63358778626
